# INITIALIZATION

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

In [0]:
# Raw to Silver renaming
RENAME_MAP = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "maintenance": "maintenance_flag"
}

# READ THE BRONZE TABLE

In [0]:
df = spark.table("workspace.bronze.erp_px_cat_g1v2_raw")

# Silver Transformations

## Trimming

In [0]:
# For trimming the whitespace on all the string columents which helps to prevent filter and join issues from messy text
for field in df.schema.fields:
  if isinstance(field.dataType, StringType):
    df = df.withColumn(field.name, trim(col(field.name)))

## Renaming

In [0]:
for old_name, new_name in RENAME_MAP.items():
  df = df.withColumnRenamed(old_name, new_name)

# Writing the Silver Table

In [0]:
# We write a silver table that is cleaned + standardized
(
    df.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("silver.erp_product_category")
)

In [0]:
%sql
-- Quick check
SELECT * FROM silver.erp_product_category